# Generalized linear models: Binary logistic regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update(
    {"mathtext.default": "regular", "figure.dpi": 300, "figure.figsize": (8, 6)}
)
import seaborn as sns
import statsmodels
import statsmodels.formula.api as smf

*Gries, chapter 5.3 (pp. 293-316)*

Let us revisit a previous exercise on teenagers' ('large' vs 'small') submission of chat conversations to a sociolinguistic research project. 
Instead of performing a chi-squared test, we will now run a binary logistic regression. Important distinctions with some simpler correlation tests are that a **binary logistic regression** model:
- includes a direction: response (output) variable versus predictor (input) variables
- can include many types of **predictor** variables, not just categorical ones (**the response variable, however, is always binary!**)
- allows us to inspect the impact of multiple predictors simultaneously
- and to include interactions between them

## Loading and inspecting the data

Load and inspect 'chat.tsv'.

In [ ]:
chat = pd.read_csv("../../datasets/chat/chat.tsv", sep="\t")
chat

##### Question: Does the assumption of data independence hold? 
> Are there repeated measurements in the data, i.e. >1 observation for 1 subject? This is important to check, since **data independence is an assumption of regression models without random effects** (as well as of many other statistical tests). 

In [ ]:
chat.subject_ID.is_unique

Now create a new binary variable: was the participant's submission to the research project large (>500 tokens of chat data) or small (<=500 tokens)? Using statsmodels, this needs to be a (numeric) binary variable for this case.

In [ ]:
chat["sub_large"] = np.where(chat.nr_tokens > 500, 1, 0)
chat

## What is a Logit?

You will often see people talk about 'logits' or 'logit link functions'. These terms come from a special curve which is often used to describe the probability of **binary outcomes**. In other words, the output *must* be $A$ or $B$, but the relative probabilities of the two outcomes is *continuous*. The function is:

$\Large\frac{1}{ 1 + e^{-x}}$

The curve looks like this:

In [ ]:
x = np.linspace(-6, 6, 1000)
y = 1 / (1 + np.exp(-x))
ax = sns.lineplot(x=x, y=y)
ax.axhline(0.5, linewidth=0.5, color="black")
ax.axvline(0, linewidth=0.5, color="black")
plt.show()

The reason logits are important is that it is common (for maths reasons) for models to output *exponential parameters* but mostly as humans we would prefer to read these are *pure probabilities* or *odds ratios*, as we have done earlier in the course. The logit lets us convert between those domains. This is also, by the way, why it is called 'logistic regression'.

As with `ols()`, the model fits a **regression line** that it will use to make predictions. Only two numbers are used to mathematically define a line: an **intercept** ('Intercept' in the output) and a **slope** for the line (the predictor coefficient in the output, here 'gendermale'). For a binary predictor, the regression line does not run through all the datapoints, but rather connects the estimated output for the predictor levels: so here, the regression line runs through the output (**chance** of a large submission) for female and male participants (which are the two levels of the predictor 'gender').

> MATHS: Recall that a straight line in log space is a curvy exponential line in 'normal' numbers

## Binary logistic regression with one binary predictor

Let's now move on to a **binary logistic regression model**. You will notice that the syntax looks a lot like what you are already used to for linear modeling with `ols()`. That makes sense, because what we will be fitting now, is just a special ('generalized') variant of linear regression, that is adapted in a way so that it makes sense for a binary (rather than a numeric) response variable. For statsmodels, there is a special class for logistic regression with the formula API called `logit`.

First, we only include gender as a predictor.

### Model fitting

Fit a model using `logit()` (for Generalized Linear Model)
- `formula`: response ~ predictor(s)
- `data`: the dataset

Then inspect the model with `summary()`. The output looks similar to the `ols()` output, but with fewer diagnostics. Because the predicted values are either `0` or `1`, it doesn't make sense to look at them for a categorial predictor. The AIC and BIC are available as methods on the fitted model, but are not presented in the summary. The $R^2$ value is not available as such for logistic models, but a similar value is shown, which you can interpret more or less the same way (low is bad, 1 is perfect).

In [ ]:
model_gender = smf.logit("sub_large ~ gender", data=chat).fit()
print(model_gender.summary())

Note that at first glance, we can already derive from the coefficients table that male participants will make $fewer$ large donations compared to female participants, as the estimate for 'gendermale' is negative. (So this means: the regression line between genderfemale and gendermale goes down, moving from the reference group to the non-reference group, as it has a negative slope). In combination with the small p-value, we can already conclude the following: boys appear to make *significantly* (p <= 0.05) fewer large donations than girls. In some analyses, this interpretation will suffice. However, sometimes, you may like to get actual probabilities for your predictor levels from the model. You can obtain these in multiple ways.

One nice trick is that you can modify the formula to base the results on a different reference level for each categorical predictor:

In [ ]:
model_gender_relevel = smf.logit(
    "sub_large ~ C(gender,Treatment(reference=1))", data=chat
).fit()
print(model_gender_relevel.summary())

The transformation function to convert a binary logistic regression model's estimates to **odds** is the **exponential function** (see also Gries p. 300):

To convert the estimated parameter and the confidence intervals to an odds ratio, we can use the `exp()` method. Unfortunately it is slightly clunky, but here we go:

In [ ]:
# we're going to do this a lot, so make a function


def get_coef_ci(
    m: statsmodels.discrete.discrete_model.BinaryResultsWrapper,
) -> pd.DataFrame:
    coefs = pd.concat([m.params, m.conf_int()], axis=1)
    coefs.columns = ["coef", "lo", "hi"]
    return pd.DataFrame(coefs)

In [ ]:
coefs = get_coef_ci(model_gender_relevel)
np.exp(coefs)

This says that `T.female` (treatment = change gender to female) makes it on average 1.39 $\times$ as likely to donate a large submission (compared to the reference level of `male`), with a 95% CI of $(1.22,1.73)$. If you look back to session 3-3, you will find that this is *exactly* the statistic we calculated with the 2x2 contingency table.

We can also extract raw probabilities (instead of relative odds). To do this we will add a 'dummy intercept' to the model, so that the results for each predictor level are listed with their 'full' value. Notice the value for `gender[male]` is now the sum of the intercept (female) and `gender[T.male]` from the first model. We call this `model_gender_fr` for 'full rank' (which refers to the size of the design matrices, but it's not important for now)

In [ ]:
model_gender_fr = smf.logit("sub_large ~ 0 + gender", data=chat).fit()
print(model_gender_fr.summary())

To convert a model value to a probability, we apply the inverse logit transformation we talked about right at the start:

$\Large\frac{1}{ 1 + e^{-x}}$


In [ ]:
# Treat a dataframe as a single variable. Wild.

c = get_coef_ci(model_gender_fr)
r = 1 / (1 + np.exp(-c))
r

With a little bit of messing around, we can make a nice effect plot:

In [ ]:
ax = sns.pointplot(x=r.index, y=r.coef)

for _, x in r.iterrows():
    ax.errorbar(
        x.name,  # x-coord (categorical)
        x.coef,  # y-coord
        x.hi - x.coef,  # err (symmetrical)
        color="orange",
        zorder=1,
        marker="|",
        capsize=5,
    )
ax.set(
    ylabel="probability",
    xlabel="gender",
    title="Logistic Regression: Probability of large donation",
)
plt.show()

<a href="https://imgflip.com/i/965h6v"><img src="https://i.imgflip.com/965h6v.jpg" title="made at imgflip.com"/></a><div>

## Binary logistic regression with a categorical predictor

Let us now add the three-level categorical variable 'education' as predictor. You will notice that the output looks a bit more complex and tricky than for a binary predictor...

In [ ]:
chat["education"] = chat.education.astype(
    pd.api.types.CategoricalDtype(
        categories=["general", "technical", "vocational"], ordered=True
    )
)

In [ ]:
model_ed = smf.logit("sub_large ~ gender + education", data=chat).fit()
print(model_ed.summary())

The output for gender can be interpreted as above: we can tell from the table that boys make significantly fewer large donations than girls.

However, the output for education is a bit more tricky, as the **p-values and estimates in the output for categorical predictors with >2 levels only relate to the predictor's levels *compared to the reference level*.** So in this table, the reference groups are 'female' for gender and 'general' for education (the first ones alphabetically, and not figuring in the output table). Therefore, the coefficient for 'gendermale' makes the comparison to the reference group 'genderfemale' - for a binary variable, that covers *all* levels of the predictor, so that is easy! But the coefficients for 'educationtechnical' and 'educationvocational' only show the comparison to 'educationgeneral' (the reference level for education), and not to each other! It is possible that 'educationtechnical' and 'educationvocational' do not sig. differ from one another - that is not shown in this table. You could figure that out by e.g. releveling the education variable and then rerunning the model and inspecting the summary again.

In [ ]:
model_ed_relevel = smf.logit(
    "sub_large ~ gender + C(education, Treatment(reference=1))", data=chat
).fit()
print(model_ed_relevel.summary())

### Is it any good?

Once again, we will check:
- Is the *model* significant (LLR p-value)
- Are the predictors *significant* (coefficient p-values)
- Did we *reduce* AIC?
- Did we *increase* (pseudo) R-squared?

In [ ]:
print(f"AIC change: {model_ed.aic - model_gender.aic:.3f}")
print(f"pseudo R-squared change: {model_ed.prsquared - model_gender.prsquared:.3f}")

> ANSWER: Yes. It's better. Both predictors are significant, and our model diagnostics have improved.

## Interaction Terms

Finally, let us include the **interaction** between our two predictors (indicated in the formula with `*`): the goal is to investigate whether the effect of one predictor on the response is affected by the other predictor in some way. E.g. the influence of gender on submission size may differ depending on the participant's educational track. Or the impact of education on submission size may be different for boys versus girls.

Fit the model and look at the summary. In the output table, you will now not only see estimates for the main effects (gender and education), but for their interaction too. An interaction is added by combining predictors with `*` rather than with `+`(which is just an addition of main effects, no interaction). In this formula language, the syntax

> response ~ predictorA * predictorB

is shorthand for

> response ~ predictorA + predictorB + predictorA:predictorB

We note that when reporting models it is considered bad practice to only include the part `predictorA:predictorB`, and thus exclude the actual main effects that create the interaction. So, **when including an interaction, always use the `*` syntax!**

In [ ]:
model_ed_interact = smf.logit("sub_large ~ gender*education", data=chat).fit()
print(model_ed_interact.summary())

We will interpret the output below, but note already that this output table is even trickier than the previous one to interpret:

> **IMPORTANT: when a (significant) interaction is included, you should *only* interpret that interaction and not the main effects on their own** (since the significant interaction literally tells you that it does not make sense to interpret the two predictors on their own). So in this output table, it would be a (common!) error to look at the p-value for gender and education, rather than only at the interaction coefficients.

In [ ]:
print(f"AIC change: {model_ed_interact.aic - model_ed.aic:.3f}")
print(
    f"pseudo R-squared change: {model_ed_interact.prsquared - model_ed.prsquared:.3f}"
)

So, once again the model improved, and the interaction is significant.

## Isolating Interaction Effects

To get the full effect, you need to include **all the factors from the model**. The baselines are `gender[female]` and `education[general]` so for women we can simply add the education level to get the interactions. For example `gender[female]` (the intercept) + `education[vocational]` adds up to exactly 0. This is just co-incidence, by the way! It turns out that exactly 50% of those women donated a large submission, and the log-odds for a model value of 0 is 1 (even odds). Spooky.

In [ ]:
chat[
    (chat.gender == "female") & (chat.education == "vocational")
].sub_large.value_counts()

To get effects for males is more complicated. To calculate the full effect for e.g. males in technical education we need to sum:
- The intercept
- adjustment from baseline for `gender[male]`
- adjustment from baseline education for `education[technical]`
- adjustment from baseline for the interaction term `gender[male]:education[technical]`

Let's extract the coefficients (and confidence intervals) from the model

In [ ]:
coefs_ed = get_coef_ci(model_ed_interact)
coefs_ed

And now we calculate that value for males in technical education by hand. You will probably agree that this is really annoying...

In [ ]:
(
    coefs_ed.loc["Intercept"]
    + coefs_ed.loc["gender[T.male]"]
    + coefs_ed.loc["education[T.technical]"]
    + coefs_ed.loc["gender[T.male]:education[T.technical]"]
)["coef"]

# Interaction Only Models

**If** we have validated a model, and convinced ourselves that the predictors are significant, **and then** established that there are also significant interactions, then it is possible to create a model with only the interactions. The only reason to do this is to be able to conveniently read the combined effects in one step, for analysis or plotting.

Here we will use the same trick from above to base the model on 0 (not using one of the coefficients as an intercept) to get a 'full rank' model that lists every coefficient explicitly.

In [ ]:
model_interaction_fr = smf.logit("sub_large ~ 0 + gender:education", data=chat).fit()
print(model_interaction_fr.summary())

The first thing to notice is that the interaction coefficients are now complete (and not broken into multiple base levels), and we can check the value for e.g. `gender[male]:education[technical]` that we calculated by hand before

In [ ]:
c = get_coef_ci(model_interaction_fr)

print(c)
print(f"\nmale + technical: {c.loc['gender[male]:education[technical]'].coef}")

Now we will convert to probabilities, and make another effect plot! This plotting is a little complicated, so don't worry too much about it. You **will** need to know how to either calculate combined interaction effects or read them from a model, but you **won't** need to plot them like this for the exam.

The table below contains the same information as the plot.

In [ ]:
# Convert from params to probabilities with inverse logit

interaction_resuilts = 1 / (1 + np.exp(-c))
interaction_resuilts["gender"] = [
    x[0] for x in interaction_resuilts.index.str.split(":")
]
interaction_resuilts["ed"] = [x[1] for x in interaction_resuilts.index.str.split(":")]
interaction_resuilts

In [ ]:
ax = sns.pointplot(
    x=interaction_resuilts.ed,
    y=interaction_resuilts.coef,
    hue=interaction_resuilts.gender,
)
for _, x in interaction_resuilts.iterrows():
    c = "steelblue"
    if x.gender == "gender[male]":
        c = "orange"
    ax.errorbar(
        x.ed,
        x.coef,
        x.hi - x.coef,
        color=c,
        zorder=1,
        linestyle="--",
        marker="|",
        capsize=5,
    )
ax.set(
    ylabel="probability",
    xlabel="education level",
    title="Logistic Regression: Probability of large donation",
)
plt.show()

##### Interpretation:
> We can see that among both boys and girls, technical students are most likely to donate a large submission. However, while for girls, general students are least likely to donate a large submission, for boys it's the vocational students who are least likely. The table also provides insight in gender differences per educational track: while the probabilities of submitting a large donation are almost identical for boys and girls in general education, a gender gap (with girls being more likely to submit a large portion of chat data) becomes apparent in technical and especially vocational education.

```
Version History

Current: v1.0.0

10/10/24: 1.0.0: first draft, BN
```